In [5]:
import numpy as np
import pandas as pd

In [6]:
t111 = pd.read_csv('./sdg_piracy_sheets/csv_data/t111.csv', index_col=[0])
t411 = pd.read_csv('./sdg_piracy_sheets/csv_data/t411.csv', index_col=[0])
t14c1 = pd.read_csv('./sdg_piracy_sheets/csv_data/t14c1.csv', index_col=[0])

In [7]:
drop_lis = ['Goal', 'Target', 'SeriesCode', 'SeriesDescription', 'FootNote']
t111.drop(drop_lis, axis=1, inplace=True)
t411.drop(drop_lis, axis=1, inplace=True)
t14c1.drop(drop_lis, axis=1, inplace=True)

In [8]:
poverty= t111[t111['Age']!='15-24']
poverty= poverty[poverty['Age']!= '15+']
poverty= poverty[poverty['Age']!= '25+']
poverty= poverty[poverty['Sex']!= 'MALE']
poverty= poverty[poverty['Age']!= 'FEMALE']

In [9]:
poverty.isnull().sum()

Indicator            0
GeoAreaCode          0
GeoAreaName          0
TimePeriod           0
Value                0
Time_Detail          0
Source               0
Age               2084
Nature               0
Reporting Type       0
Sex               2084
Units                0
dtype: int64

In [10]:
drop_lis = ['Age', 'Sex', 'Time_Detail', 'Source', 'Nature', 'Reporting Type', 'Units']
poverty.drop(drop_lis, axis=1, inplace=True)

In [11]:
poverty.rename(columns={'Value':'poverty_percentage'}, inplace=True)

In [12]:
poverty.shape

(2084, 5)

In [13]:
#https://stackoverflow.com/questions/65817843/how-to-create-new-rows-with-missing-years-and-populate-them-with-current-rows
poverty = (poverty.set_index('TimePeriod').groupby('GeoAreaCode')
   .apply(lambda x: x.reindex(np.arange(2000, 2022)).ffill())
   .reset_index('GeoAreaCode',drop=True)
   .reset_index())
poverty.dropna(inplace=True)

In [14]:
poverty = poverty[poverty['TimePeriod'] > 2009]

In [15]:
poverty.to_csv('./datasets/poverty.csv')

In [16]:
t411= t411[t411['Sex']!= 'MALE']
t411= t411[t411['Sex']!= 'FEMALE']
t411= t411[t411['Education level']== 'LOWSEC']
t411m = t411[t411['Type of skill']== 'SKILL_MATH']
t411r = t411[t411['Type of skill']== 'SKILL_READ']

In [17]:
def year_filler(df, low=2000, high=2022):
    df = (df.set_index('TimePeriod').groupby('GeoAreaCode').apply(lambda x: x.reindex(np.arange(low, high)).ffill()).reset_index('GeoAreaCode',drop=True).reset_index())
    df.dropna(inplace=True)
    return df

In [18]:
t411m = year_filler(t411m)
t411r = year_filler(t411r)

In [19]:
t411m['join_key'] = list(zip(t411m['GeoAreaCode'], t411m['TimePeriod']))
math = t411m[['join_key', 'Value']]

t411r['join_key'] = list(zip(t411r['GeoAreaCode'], t411r['TimePeriod']))
reading = t411r[['join_key', 'Value']]

math.rename(columns={'Value': 'math_benchmark_percent'}, inplace=True)
reading.rename(columns={'Value': 'reading_benchmark_percent'}, inplace=True)

C:\Users\Matt\anaconda3\lib\site-packages\pandas\core\frame.py:5034: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [20]:
math.to_csv('./datasets/math.csv')
reading.to_csv('./datasets/reading.csv')

In [23]:
sea_law_adoption = t14c1[['GeoAreaCode', 'Value']]
sea_law_adoption.rename(columns={'Value': 'sea_law_adoption', 'GeoAreaCode': 'country_code'}, inplace=True)

In [24]:
sea_law_adoption.to_csv('./datasets/sea_law_adoption.csv')